# 1. Set Up Notebook and Load Data 

In [1]:
# PYTHON Imports 
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pathlib import Path
import glob
import os
import ipywidgets as widgets
from IPython.display import clear_output
import sys
import time
import json
import seaborn as sns
import fnmatch
# ML Imports
import tensorflow as tf
from tensorflow.keras import layers, models, losses
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Reshape
from tensorflow.keras.models import Model, Sequential
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard
# ASTROPHY Imports
import astropy 
from astropy.table import Table
from astropy.io import fits
from sherpa.astro import ui
# CIAO Imports
import ciao_contrib.runtool
from ciao_contrib.runtool import *
# CUSTOM Imports
from data_extraction_functions import *
from data_exploration_functions import *
from data_representation_functions import *

# Specify global path
global_path = '/Users/steven/Library/Mobile Documents/com~apple~CloudDocs/0-CfA/4-Data/Datasets'
global_folders = list_folders_fun(global_path)

# Select dataset
set_widget = widgets.Dropdown(options=global_folders[:],value=global_folders[0],description='Set :',disabled=False); set_widget

2023-03-16 13:43:17.328017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dropdown(description='Set :', options=('All', 'Bona'), value='All')

In [54]:
# Set ID
set_id = set_widget.value
# Select Input
files = os.listdir(f'{global_path}/{set_id}/')
input_files = [f for f in files if fnmatch.fnmatch(f, 'hist2D*')]
input_widget = widgets.Dropdown(options=input_files[:],value=input_files[0],description='TSNE File :',disabled=False); input_widget

Dropdown(description='TSNE File :', options=('hist2D-All-nE20-nt30-normnone.pkl', 'hist2D-All-nE24-nt32-normno…

In [55]:
# Load the DataFrame from the CSV file
input_file = input_widget.value
# Load histogram dictionary
with open(f'{global_path}/{set_id}/{input_file}', 'rb') as f:
    hist_dict = pickle.load(f)
# Flatten histograms in the dictionary and get IDs
ids = hist_dict.keys()
histograms = hist_dict.values()
features = np.array([np.array(h) for h in histograms])
#features[np.isnan(features)] = 0.0
# Load properties
df_properties_input = pd.read_csv(f'{global_path}/{set_id}/properties-input-{set_id}.csv')
df_properties_input = df_properties_input[df_properties_input['obsreg_id'].isin(list(ids))]
df_properties = df_properties_input.drop_duplicates('obsreg_id', keep='first').reset_index(drop=True)
# Print eventfiles and properties number of IDs
print("Number of Features: ", len(features))
print("Number of Property Sets: ", len(df_properties))

df_properties.head()

Number of Features:  82283
Number of Property Sets:  82283


,cnts_aper_b,cnts_aperbkg_b,src_cnts_aper_b,flux_aper_b,hard_hm,hard_hs,hard_ms,var_prob_b,var_prob_h,var_prob_m,var_prob_s,obsreg_id
0,66,38,68.744294,5.470823e-13,0.223610,0.745784,0.618364,0.310792,0.117484,0.675801,NaN,10003_2
1,53,20,55.350611,5.470823e-13,0.398501,0.488445,0.094941,0.524820,0.127010,0.676527,0.485168,10004_1
2,39,28,41.900915,4.311669e-13,0.873204,0.999375,0.999375,0.765525,0.888159,NaN,NaN,10018_1
3,144,1354,94.226698,5.097861e-14,0.058713,-0.103685,-0.162399,0.105254,0.252282,0.650681,0.283007,10025_72
4,202,2417,118.341424,7.185242e-14,0.339788,0.678326,0.413492,NaN,0.650170,NaN,NaN,10025_74


# 2. Data Prep

In [56]:
# Prepare Data
df_label = df_properties.copy()
# mask_nonan = df_label.notna().all(axis=1)
# index_nonan = list(df_label.notna().all(axis=1).index[df_label.notna().all(axis=1)])
# df_label = df_label[mask_nonan]
ID = df_label['obsreg_id'].values
# X = np.array([features[i] for i in index_nonan])
X = features
# Split into training and test data
X_train, X_test, ID_train, ID_test = train_test_split(X, ID, test_size=0.2, random_state=42)
# Define Input Shape
in_shape = X[0].shape
in_shape_1 = in_shape[0]
in_shape_2 = in_shape[1]
# Reshape data to 4D tensor for use with CNN
# X_train = X_train.reshape(X_train.shape[0], in_shape_1, in_shape_2, 1)
# X_test = X_test.reshape(X_test.shape[0], in_shape_1, in_shape_2, 1)
# X_train = tf.expand_dims(X_train, 3)
# X_test = tf.expand_dims(X_test, 3)

# 3. Autoencoder Model

In [59]:
batch_size = 25

# Encoder
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(in_shape_1, in_shape_2, 1)))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))

# Decoder
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.summary()

#Fit the model. 
history = model.fit(
        X_train, X_train,
        steps_per_epoch= 500 // batch_size,
        epochs=1000,
        validation_data=(X_test,X_test),
        validation_steps=75 // batch_size,
        shuffle = True)


Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_129 (Conv2D)         (None, 32, 24, 64)        640       
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 16, 12, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_130 (Conv2D)         (None, 16, 12, 32)        18464     
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 8, 6, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_131 (Conv2D)         (None, 8, 6, 16)          4624      
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 4, 3, 16)       

2023-03-14 15:38:54.954712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-14 15:38:56.330650: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fd1e69f3380
2023-03-14 15:38:56.330796: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fd1e69f3380


NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_12' defined at (most recent call last):
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/anaco

In [34]:
# Define Autoencoder Model
latent_dim = 100

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Input(in_shape_1,in_shape_2),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu')
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(in_shape_1*in_shape_2, activation='sigmoid'),
      layers.Reshape((in_shape_1, in_shape_2))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim) 

# Compile Autoencoder Model
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

# Train Autoencoder Model
autoencoder.fit(X_train, X_train,
                epochs=20,
                shuffle=True,
                validation_data=(X_test, X_test))

# autoencoder.fit(X_train, X_train,
#                 epochs=50,
#                 batch_size=32,
#                 shuffle=True,
#                 validation_data=(X_train, X_train),
#                 callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Epoch 1/20


ValueError: in user code:

    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/q1/jbx7qj7s0dxbh7nv02nfvc680000gn/T/__autograph_generated_fileoouzqmtq